In [1]:
import numpy
import scipy.stats
from scipy.integrate import quad, dblquad

In [2]:
from timeit import default_timer as timer

In [3]:
CHEB_NODES_NB = 10
LOW_BOUND = -10.
UPP_BOUND = 10.

In [4]:
N = 10
M = 2000000

# Gaussian expectation vector
Mu = numpy.zeros(N)

# Gaussian variance-covariance matrix
Sigma = numpy.array([[ 2.11,  0.37, -0.42, -0.2 , -1.73, -0.54,  0.42,  0.81, -0.2 , -0.94],
                     [ 0.37,  1.78, -0.45, -0.25, -1.73,  0.04,  0.35,  0.38, -0.91, -0.48],
                     [-0.42, -0.45,  0.87,  0.09,  0.4 , -0.05, -0.25,  0.06,  0.16,  0.45],
                     [-0.2 , -0.25,  0.09,  0.19,  0.38, -0.04, -0.11, -0.04,  0.22,  0.17],
                     [-1.73, -1.73,  0.4 ,  0.38,  5.3 ,  0.61,  0.46,  0.26,  1.74,  1.46],
                     [-0.54,  0.04, -0.05, -0.04,  0.61,  0.53,  0.1 , -0.3 , -0.1 ,  0.17],
                     [ 0.42,  0.35, -0.25, -0.11,  0.46,  0.1 ,  0.91,  0.51, -0.17, -0.25],
                     [ 0.81,  0.38,  0.06, -0.04,  0.26, -0.3 ,  0.51,  1.55,  0.49,  0.08],
                     [-0.2 , -0.91,  0.16,  0.22,  1.74, -0.1 , -0.17,  0.49,  1.33,  0.65],
                     [-0.94, -0.48,  0.45,  0.17,  1.46,  0.17, -0.25,  0.08,  0.65,  0.88]])

print "Mu = %s" % Mu
print "Sigma = \n %s" % Sigma

ts_rv = timer()
rv = scipy.stats.multivariate_normal (mean=Mu, cov=Sigma, allow_singular=True)
X = rv.rvs(size=M)
te_rv = timer()

ctp = te_rv - ts_rv

Mu = [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Sigma = 
 [[ 2.11  0.37 -0.42 -0.2  -1.73 -0.54  0.42  0.81 -0.2  -0.94]
 [ 0.37  1.78 -0.45 -0.25 -1.73  0.04  0.35  0.38 -0.91 -0.48]
 [-0.42 -0.45  0.87  0.09  0.4  -0.05 -0.25  0.06  0.16  0.45]
 [-0.2  -0.25  0.09  0.19  0.38 -0.04 -0.11 -0.04  0.22  0.17]
 [-1.73 -1.73  0.4   0.38  5.3   0.61  0.46  0.26  1.74  1.46]
 [-0.54  0.04 -0.05 -0.04  0.61  0.53  0.1  -0.3  -0.1   0.17]
 [ 0.42  0.35 -0.25 -0.11  0.46  0.1   0.91  0.51 -0.17 -0.25]
 [ 0.81  0.38  0.06 -0.04  0.26 -0.3   0.51  1.55  0.49  0.08]
 [-0.2  -0.91  0.16  0.22  1.74 -0.1  -0.17  0.49  1.33  0.65]
 [-0.94 -0.48  0.45  0.17  1.46  0.17 -0.25  0.08  0.65  0.88]]


# Link between paper and code

\begin{equation*}
\ell(x) = \sum_k x_k + \frac{1}{2} \sum_k (x_k^+)^2 + \sum_{j < k} x_j^ + x_k^+
\end{equation*}

We need to solve the following system:
    
\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ \mathbb{E} \left( \left(X_k - m_k^* \right)^+ \right) 
+ \sum_{j \neq k} \mathbb{E} \left( \left(X_j - m_j^* \right)^+ 1_{X_k > m_k^*} \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \mathbb{E} \left( X_k - m_k^* \right) 
+ \sum_k \frac{1}{2} \mathbb{E} \left( \left[ \left( X_k - m_k^* \right)^+ \right]^2 \right) 
+ \sum_{j < k} \mathbb{E} \left( \left( X_j - m_j^* \right)^+ \left( X_k - m_k^* \right)^+ \right)
= c
\end{cases}
\end{equation*}

Definitions from the paper:

\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ f_k \left( m_k^* \right) 
+ \sum_{j \neq k} l_{j, k} \left( m_j^*, m_k^* \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \left( e_k \left( m_k^* \right) 
+ \frac{1}{2} g_k \left( m_k^* \right) 
+ \sum_{j < k} h \left( m_j^*, m_k^* \right)
\right) 
= c
\end{cases}
\end{equation*}

In [5]:
def e(index, x):
    mu = Mu[index]
    
    return mu - x

In [6]:
def f(k, x):
    global X
    Xk = X[:, k]
    Xkmx = Xk - x    
    
    Xkmxp = numpy.maximum(Xkmx, 0.)
    
    return numpy.mean(Xkmxp)

In [7]:
def g(k, x):
    global X
    Xk = X[:, k]
    Xkmx = Xk - x    
    
    Xkmxp = numpy.maximum(Xkmx, 0.)
    
    Xkmxp2 = numpy.square(Xkmxp)
    
    return numpy.mean(Xkmxp2)

In [8]:
def h(j, k, x, y):
    global X
    Xjk = X[:, [j, k]]
    
    m = numpy.array([x, y])
    zero = numpy.zeros(2)
        
    Xmm = numpy.subtract(Xjk, m)
    Xmmp = numpy.maximum(Xmm, zero)
    
    prod = numpy.cumprod(Xmmp, axis=1)[:, -1]
    return numpy.mean(prod)

In [9]:
def l(j, k, x, y):
    global X
    Xjk = X[:, [j, k]]
    
    m = numpy.array([x, y])
    zero = numpy.zeros(2)
        
    Xmm = numpy.subtract(Xjk, m)
    Xmmp = numpy.maximum(Xmm, zero)
    
    # We use the fact that sgn(0) = 0
    Xmmp[:, 1] = numpy.sign(Xmmp[:, 1])

    prod = numpy.cumprod(Xmmp, axis=1)[:, -1]
    return numpy.mean(prod)    

In [10]:
class WrapperFunc(object):
    def __init__(self, func, indexes):
        self._i = indexes
        self._f = func
        
    def __call__(self, *x):
        tmp_args = self._i + list(x)
        return self._f(*tmp_args)

## Put the variables and functions to the remote engines

In [11]:
import sys

sys.path.append("../../../lib")

In [12]:
from tchebychev import TchebychevFun

def init_funcs():
    from timeit import default_timer as timer
    
    funcs = dict()
    interpol_time = 0.
    
    for k in range(N):
        ek = WrapperFunc(e, [k])
        tic = timer()
        ek = TchebychevFun(ek, [LOW_BOUND], [UPP_BOUND], [CHEB_NODES_NB])
        interpol_time += timer() - tic
        
        fk = WrapperFunc(f, [k])
        tic = timer()
        fk = TchebychevFun(fk, [LOW_BOUND], [UPP_BOUND], [CHEB_NODES_NB])
        interpol_time += timer() - tic
        
        gk = WrapperFunc(g, [k])
        tic = timer()
        gk = TchebychevFun(gk, [LOW_BOUND], [UPP_BOUND], [CHEB_NODES_NB])
        interpol_time += timer() - tic
    
        hk = dict()
        lk = dict()
        for j in range(N):
            if j < k:
                hjk = WrapperFunc(h, [j, k])
                tic = timer()
                hjk = TchebychevFun(hjk, [LOW_BOUND, LOW_BOUND], [UPP_BOUND, UPP_BOUND], [CHEB_NODES_NB, CHEB_NODES_NB])
                interpol_time += timer() - tic                
                hk[j] = hjk
                
            if j != k:
                ljk = WrapperFunc(l, [j, k])
                tic = timer()
                ljk = TchebychevFun(ljk, [LOW_BOUND, LOW_BOUND], [UPP_BOUND, UPP_BOUND], [CHEB_NODES_NB, CHEB_NODES_NB])
                interpol_time += timer() - tic                
                lk[j] = ljk
    
        funcs[k] = {"e": ek, "f": fk, "g": gk, 
                    "hk": hk, "lk": lk}
        
    global ctp
    ctp += interpol_time
        
    return funcs

In [13]:
def prepare_parameterized_funcs(dict_funcs):
    res = dict()
        
    for k, v in dict_funcs.iteritems():
        for kk, vv in v.iteritems():
            if isinstance(vv, dict):
                for j, f in vv.iteritems():
                    key = "%s%s%s" % (kk[:-1], j, k)
                    res[key] = (f, j, k)
            else:
                key = "%s%s" % (kk, k)
                res[key] = (vv, k)
        
    return res

In [14]:
def run_par_func(param):
    f = param[0]
    vm = []
    for i_m in param[1:]:
        vm.append(M[i_m])
    
    return f(*vm)

In [15]:
funcs = init_funcs()
flatten_funcs = prepare_parameterized_funcs(funcs)

In [16]:
from scipy.optimize import fsolve

def optimize(c, **kwargs):
    global N
    
    __results = {key: dict() for key in "efghl"}
        
    def fill_results(lbl, res):
        fname = lbl[0]
        k = int(lbl[-1])

        if len(lbl) == 3:
            j = int(lbl[1])
            if k not in __results[fname]:
                __results[fname][k] = dict()
                
            __results[fname][k][j] = res
        else:
            __results[fname][k] = res
    
    def objective(v_x):
        _lambda = v_x[-1]
        
        # Local version
        global M
        M = numpy.clip(v_x, LOW_BOUND, UPP_BOUND)
        f_res = map(run_par_func, flatten_funcs.values())
        
        for lbl, fx in zip(flatten_funcs.keys(), f_res):
            fill_results(lbl, fx)
            
        last = 0.
        res = numpy.zeros(N + 1)
        
        for k in range(N):
            res_k = 1 + __results["f"][k]
            for j, v in __results["l"][k].iteritems():
                res_k += v
                
            res[k] = (_lambda * res_k - 1.)**2
            
            last += __results["e"][k] + .5 * __results["g"][k]
            if k in __results["h"]:
                for j, v in __results["h"][k].iteritems():
                    last += v
                    
        res[-1] = (last - c)**2
        return res
    
    guess = numpy.zeros(N + 1)
    if 'guess' in kwargs and kwargs['guess'] is not None:
        guess = kwargs.pop('guess')
        
    return fsolve(objective, guess, **kwargs) 

In [17]:
c = 1.

guess = numpy.zeros(N+1)

In [18]:
tic = timer()

continue_ = True
__i = 1
l_N = 0
while continue_:
    print "iteration nb %i" % __i
    optim_result = optimize(c, guess=guess, maxfev=1000, full_output=True)
    continue_ = optim_result[2] != 1
    if continue_:
        guess = optim_result[0]
        __i += 1
        l_N += optim_result[1]["nfev"]
        print
    
toc_m_tic = timer() - tic

print optim_result[-1]

iteration nb 1
The solution converged.


In [19]:
m_star = optim_result[0][:-1]
lb_star = optim_result[0][-1]
cto = toc_m_tic

print "Time of optimization: %.2f sec" % (cto)
print "m star = %s" % m_star
print "lambda star = %s" % lb_star

Time of optimization: 51.47 sec
m star = [ 0.18235808  0.10592587  0.27206959  0.21119392  1.1878535   0.19777244
  0.52933814  0.77628965  0.59595745  0.47692109]
lambda star = 0.441614257325


In [20]:
l_N += optim_result[1]["nfev"]

print "l(%i) = %i" % (N, l_N)
print "function evaluated at the output = %s" % optim_result[1]["fvec"]

l(10) = 120
function evaluated at the output = [  3.45439444e-23   3.45504700e-23   3.45948600e-23   3.45987782e-23
   3.47884193e-23   3.45961660e-23   3.46379717e-23   3.47242757e-23
   3.46654204e-23   3.46431992e-23   2.92597543e-23]


### Just need to copy the results in Excel:

In [21]:
print m_star
print
print "%f \n %i \n %.2f \n %.2f" % (lb_star, l_N, ctp, cto)

[ 0.18235808  0.10592587  0.27206959  0.21119392  1.1878535   0.19777244
  0.52933814  0.77628965  0.59595745  0.47692109]

0.441614 
 120 
 3533.13 
 51.47
